In [ ]:
import pandas as pd
import datetime as dt
import scipy.stats as st 
import numpy as np

In [ ]:
def Black_Scholes(S,K,r,T,vol,otype="call"):
    d1 = (np.log(S/K) + (r + 0.5*vol**2) * T) / (vol*np.sqrt(T))
    d2 = d1 - vol*np.sqrt(T)
    if otype == "call": 
        call = S*st.norm.cdf(d1) - K*np.exp(-r*T)*st.norm.cdf(d2)
        return call
    else: 
        put = K*np.exp(-r*T)*st.norm.cdf(-d2) - S*st.norm.cdf(-d1)
        return put 

# Pricing an American Option with the CRR Model

In this small presentation, I will show how to price an **American option** through the **Cox–Ross–Rubinstein (CRR)** binomial model.  
As a starting point, we will evaluate a standard European Call, which will trace the roots of our approach.

---

### Market setup

Let us consider the market  
$$\mu = \left(S^0, S^1\right),
$$
where $S^0$ is the risk-free asset and $S^1$ is the risky asset.  Consider a portfolio $\Phi = (\phi^0, \phi^1)$, where $\phi^0$ is the holding in the risk-free asset and $\phi^1$ the holding in the risky asset.  

The portfolio value is:
$$
V_t(\Phi) = S_t^0 \, \phi_t^0 + S_t^1 \, \phi_t^1.
$$

---

### Risk-free normalization and self-financing condition

A common step is **risk-free normalization**: if $t=0$ is the inception date, we set
$$
S_0^0 = 1.
$$

Assuming the portfolio is **self-financing**, we have:
$$
V_t(\Phi) = \phi_{t-1}^0 (1+r) + \phi_{t-1}^1 S_t^1,
$$
meaning that changes in portfolio value are due only to price movements, without additional cash injection or withdrawal.

---

### The one-step binomial model

Consider a one-period model, with a discrete process $\{S_t^1\}_{t=0}^T$ such that
$$
S_T^1 : \Omega \longrightarrow \mathbb{R}^+.
$$

There are two possible states:
- $u$ : "Upward movement"
- $d$ : "Downward movement"

Thus:
- $S_t^{1,u}$ denotes the stock price at time $t$ if an up move occurs,
- $S_t^{1,d}$ denotes the stock price at time $t$ if a down move occurs.

---

### Option payoff and tree construction

Let $X$ be an option written on $S^1$. At the next step, the payoff can take only two values:
$$
X_t^u \quad \text{or} \quad X_t^d.
$$

The CRR idea is:
1. Construct the **binomial tree** from $t=0$ up to maturity $T$.  
2. Compute the option payoffs at maturity.  
3. Use **backward induction under the risk-neutral probability** to recursively compute the option value at earlier nodes.  

This way, we obtain the “fair” option price today. For an **American option**, at each step we also check the possibility of early exercise:
$$
V_t = \max \{ \text{intrinsic value at } t, \ \text{continuation value} \}.
$$

---


Let us explore the idea behind this approach that relies on the numeraire choice. 
Consider the following ratio: 
$$
\frac{\mathbb{E}_{\mathbb{P}}\left[S_{t+1}\right]}{S_t}
= \frac{P \cdot S_t u + (1-P)\cdot S_t d}{S_t}
= P u + (1-P)d,
$$
where $P$ is the physical probability of up. We add and subtract $(1+r)$:
$$
\frac{\mathbb{E}_{\mathbb{P}}\left[S_{t+1}\right]}{S_t}
= (1+r) + \Big( P(u-(1+r)) + (1-P)(d-(1+r)) \Big).
$$
Hence:
$$
\frac{\mathbb{E}_{\mathbb{P}}\left[S_{t+1}\right]}{S_t} = (1+r) + \text{risk premia}.
$$

---

### Martingale approach
The idea is that under the **risk-neutral measure** $\mathbb{Q}$, the risk premia vanishes so that asset returns equal the risk-free rate. This yields a *rational price* independent of subjective beliefs. In other words, discounted prices are martingales under the numeraire $(1+r)$. 

Define the discounted stock:
$$
S_{t+1}^* = \frac{S_{t+1}}{1+r}.
$$
The martingale condition states:
$$
\mathbb{E}^{\mathbb{Q}}\!\left[S_{t+1}^* \mid \mathcal{F}_t\right] = S_t.
$$

Introducing a multiplicative model so that $S_{t+1}^u = S_t \cdot u$ and the same for $d$ we get : 
$$
\mathbb{E}^{\mathbb{Q}}\!\left[\frac{S_{t+1}}{1+r} \mid \mathcal{F}_t\right]
= \frac{q \, S_t u + (1-q)\, S_t d}{1+r} = S_t.
$$

---

### Risk-neutral probabilities
Rearranging:
$$
q(u-d) + d = 1+r,
$$
so that
$$
\boxed{\,q = \frac{1+r - d}{u-d}\,}.
$$

---

### Comparison with the physical measure
Under $\mathbb{P}$:
$$
\frac{\mathbb{E}_{\mathbb{P}}[S_{t+1}]}{S_t} = P u + (1-P)d.
$$

Under $\mathbb{Q}$:
$$
\frac{\mathbb{E}_{\mathbb{Q}}[S_{t+1}]}{S_t} = q u + (1-q)d = 1+r.
$$

So the **risk premium** is
$$
\text{Risk premium} 
= \frac{\mathbb{E}_{\mathbb{P}}[S_{t+1}]}{S_t} - (1+r)
= (P-q)(u-d).
$$
---

### Conclusion
If we work under $\mathbb{Q}$, i.e. set $P=q$, then the expected return equals exactly $1+r$, so the risk premium is zero. This is why choosing the bank account $(1+r)$ as numeraire is convenient: it wipes off the market risk premium and ensures martingale pricing.

Finally, in the Cox–Ross–Rubinstein parametrization one often sets: 
$$u = e^{\sigma \sqrt{\Delta t}}, \qquad d = \frac{1}{u},
$$
so that the binomial model converges to the Black–Scholes model in continuous time. For further details see Hull "Options, Futures & Other Derivatives"


---
## European case

In [ ]:
def Bin_tree(S0, K, r, T, vol, q , N, otype = "call") : 

    #--------------------------------------------------

    # S0 = Stock price at time 0 
    # K = Strike Price 
    # r = risk-free rate 
    # T = time to maturity (Years)
    # vol = Stock log-returns volatility
    # q = Dividend yield 
    
     #------------------Parameters----------------------

    delta_t = T / N 

    u = np.exp(vol * np.sqrt(delta_t))   # Up Movement 
    d = 1/u                              # Down Movement

    # Distinction between dividend or no dividend 
    
    if q == 0 : 
        P = (np.exp(r * delta_t) - d) / (u - d)
    else: 
        P = (np.exp((r - q)* delta_t) - d ) / (u - d)
 
    #-----------------Terminal Values--------------------

    j = np.arange(N+1)          # 1D array storing different coefficients for u 
    S = S0 * u**j * d**(N-j)    # Construction of the last period values for S 

    if otype == "call" : 
        Val = np.maximum(S - K, 0)
    elif otype == "put": 
        Val = np.maximum(K - S, 0)
    else: 
        raise ValueError("Input must be 'call' or 'put'")

    for i in range(N,0 ,-1) :    # Backward induction 

        Val = np.exp(-r * delta_t) * (P * Val[1:1+i] + (1-P) * Val[0:i])

    return Val[0]

The last loop is the **core of the pricing algorithm**. At each iteration we step one period backwards in the binomial tree: for every node we compute the expected option value under the **risk-neutral measure**, discounted at the risk-free rate. Formally, this replaces the values at the $i$-th step (`Val[0:i]`) with their risk-neutral expectation using the values from the next step (`Val[1:i+1]`).  We repeat this backward induction until we arrive at the initial node ($t=0$), which gives us the arbitrage-free price of the option today.  

For efficiency, the vector `Val` is overwritten at each iteration, so we only store the relevant values instead of building the entire tree. This makes the algorithm fast and memory-efficient. 

To check if the algorithm works fine I will show the convergence for very large N that is to say for many steps of the CRR to Black-Scholes:


In [ ]:
S0 = 100 
K = 100 
r = 0.02
vol = 0.2 
q = 0 
T = 1 

call_BS = Black_Scholes(S0, K , r , T , vol , otype= "call")
call_CRR = Bin_tree(S0, K , r , T , vol , q = 0, N = 2000 , otype="call")

print("The call price difference is : " f'',call_BS - call_CRR)

put_BS = Black_Scholes(S0, K , r , T , vol , otype= "put")
put_CRR = Bin_tree(S0, K , r , T , vol , q = 0, N = 2000 , otype="put")

print("The put price difference is : " f'',put_BS - put_CRR)

---
## American case 

In [ ]:
def Bin_tree(S0, K, r, T, vol, q , N, otype , american = False) : 

    #------------------Arguments-----------------------

    # S0 = Stock price at time 0 
    # K = Strike Price 
    # r = risk-free rate 
    # T = time to maturity (Years)
    # vol = Stock log-returns volatility
    # q = Dividend yield 
    
    #------------------Parameters------------------------

    delta_t = T / N 

    u = np.exp(vol * np.sqrt(delta_t))   # Up Movement 
    d = 1/u                              # Down Movement

    # **Distinction between dividend or no dividend**

    if q == 0 : 
        P = (np.exp(r * delta_t) - d) / (u - d)
    else: 
        P = (np.exp((r - q)* delta_t) - d ) / (u - d)

    #-----------------Terminal Values--------------------
    
    j = np.arange(N+1)          # 1D array storing different coefficients for u 
    S = S0 * u**j * d**(N-j)    # Construction of the last period values for S 

    if otype == "call" : 
        Val = np.maximum(S - K, 0)
    elif otype == "put": 
        Val = np.maximum(K - S, 0)
    else: 
        raise ValueError("Input must be 'call' or 'put'")
    
    #---------------Backward Induction------------------

    for i in range(N, 0 , -1) : 

        Continuation_Val = np.exp( -r * delta_t) * (P * Val[1:1+i] + (1-P)  * Val[0:i])     # Option Value if we wait till the next period 

    # **American case**

        if american == True :

            j_current = np.arange(i)
            S_current = S0 * u**j_current * d**(i- j_current -1)

            if otype == "call" : 
                Val_Imm = np.maximum(S_current - K , 0)

            elif otype == "put" : 
                Val_Imm = np.maximum(K - S_current , 0)

            Val = np.maximum(Continuation_Val, Val_Imm)
    
    # **European case**

        elif american == False :

            Val = Continuation_Val 

        else: 
            raise ValueError("american must be 'True' or 'False'")
        

    return Val[0]

This version of `Bin_tree` extends the European option pricer to also handle **American options**.  

The overall structure is unchanged: compute parameters (`u, d, P`), build terminal stock values, set payoffs at maturity, and apply backward induction.  

**Main differences from the European-only version:**
- An extra argument `american` determines whether early exercise is allowed.  
- At each step of backward induction, the code computes:
  - `Continuation_Val`: the discounted expected value (as in the European case).  
  - `Val_Imm`: the immediate exercise value based on current stock prices.  
- For **American options**, the node value is the maximum of continuation and immediate exercise.  
- For **European options**, the node value is just the continuation value (same as before).  

Thus, the only additional feature is the early-exercise check, which ensures correct pricing of American-style derivatives. 

--- 
## Volatility Surface construction 

To calculate the implied volatilities needed to plot the volatility surface, the Newton-Raphson method is often used due to its fast convergence. This method requires the first derivative of the objective function, which is typically the difference between the model and market option prices. Since an analytical formula for this derivative is not always available, it is commonly approximated using a numerical technique like the **Finite difference method**, derived from Taylor series expansions. The central difference formula is:

$$ f^{\prime}(x)\approx\frac{f^{}(x+h)-f^{}(x-h)}{2h} $$

A key weakness of Newton-Raphson is its instability when the first derivative, $f'(x)$, is near zero. This causes the update step, $x_{n+1} = x_n - f(x_n)/f'(x_n)$, to become very large and potentially diverge.
To ensure **robust convergence**, a safeguarded approach is implemented. A minimum threshold for the derivative is set. If the derivative falls below this threshold, the algorithm switches to the **bisection method**. Although bisection converges slower than Newton-Raphson, it guarantees convergence to a root provided the initial interval brackets it. This hybrid approach combines speed with reliability. See https://www.interactivebrokers.com/campus/ibkr-quant-news/implied-volatility-formulation-computation-and-robust-numerical-methods/ also for a good explanation of this combined approach. 

In [ ]:
def Hybrid(S0, K, r, T, MV_option, q , vol_guess, N, Z, otype, american , error=1e-4 , epsilon = 1e-6): 

    #------------------Arguments-----------------------

    # S0 = Stock price at time 0 
    # K = Strike Price 
    # r = risk-free rate 
    # T = time to maturity (Years)
    # vol = Stock log-returns volatility
    # q = Dividend yield 

    #------------------Parameters------------------------

    sigma = vol_guess
    low , high = 1e-5 , 5 


    #----------------Newton-Raphson----------------------

    for _ in range(Z) : 

        Loss = ( Bin_tree(S0, K, r, T, sigma, q , N, otype , american) - MV_option )

        if abs(Loss) < epsilon: 
            break 
        
        funct_plus = Bin_tree(S0, K, r, T, sigma + error , q , N, otype , american)
        funct_min = Bin_tree(S0, K, r, T, sigma - error, q , N, otype , american)

        der = ( funct_plus - funct_min ) / ( 2*error )      # Finite difference method

    #----------------Bisection Method---------------------
    
        if abs(der) < 1e-8  :
            
            sigma_new = ( low + high ) / 2 

            if Loss > 0 : 
                high = sigma_new
            else : 
                low = sigma_new

            sigma = sigma_new

        else : 
            sigma_new = sigma - Loss / der 

            sigma = sigma_new
            sigma_new = max(low, min(high, sigma_new))


    return sigma 

Before running the implied volatility solver we should check for the Absence of Arbitrage Opportunities to ensure the right convergence of Newton-Raphson. 

---
## Plot the volatility Surface 

In [ ]:
Moneyness = []
Time_to_maturity = []
Impl_vol = []

from scipy.interpolate import griddata
import plotly.graph_objects as go

X = Moneyness
Y = Time_to_maturity
Z = Impl_vol

xi = np.linspace(min(X) , max(X) , 50)
yi = np.linspace(min(Y) , max(Y) , 50)
Xi , Yi = np.meshgrid(xi,yi)
Zi = griddata((X,Y), Z , (Xi,Yi) , method = "linear")

# Create interactive surface plot
fig = go.Figure(data=[go.Surface(
    x=Xi, y=Yi, z=Zi,
    colorscale="Plasma",
    opacity=0.9
)])

fig.update_layout(
    title = "American call Vol Surface",
    scene=dict(
        xaxis_title="Strike",
        yaxis_title="Time to Maturity (Days)",
        zaxis_title="Implied Volatility",
    ),
    width=1000,
    height=800
)



# Show the figure explicitly
fig.show(renderer="browser")   # if it doesn't show, try "browser" or "vscode"